In [5]:
# Use base python environment
!pip install statsmodels stargazer --q

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load data
df = pd.read_stata("assignment7.dta")

# Keep Great Britain only
df = df[df["nireland"] == 0].copy()

# Generate polynomial controls
df["y1"] = df["yobirth"] - df["yobirth"].mean()
df["y2"] = df["y1"]**2
df["y3"] = df["y1"]**3
df["y4"] = df["y1"]**4

df["a1"] = df["age"] - df["age"].mean()
df["a2"] = df["a1"]**2
df["a3"] = df["a1"]**3
df["a4"] = df["a1"]**4


def reg(formula, df=df):
    """
    Runs WLS with clustering, ensuring that weights and cluster
    groups match the rows used by the regression.
    """

    # First run OLS formula to determine which observations are used
    temp = smf.ols(formula, data=df).fit()
    used = temp.model.data.row_labels   # index of rows kept

    # Filter weights + cluster to match used rows
    df_used = df.loc[used]

    # Now run WLS with aligned weights
    model = smf.wls(formula, data=df_used, weights=df_used["wght"]).fit(
        cov_type="cluster", cov_kwds={"groups": df_used["yobirth"]}
    )

    return model

# -----------------------------------------------------
# Run all Table 1 regressions
# -----------------------------------------------------
m1 = reg("agelfted ~ drop15 + y1 + y2 + y3 + y4")
m2 = reg("agelfted ~ drop15 + y1 + y2 + y3 + y4 + a1 + a2 + a3 + a4")
m3 = reg("agelfted ~ drop15 + y1 + y2 + y3 + y4 + C(age)")

m4 = reg("learn ~ drop15 + y1 + y2 + y3 + y4")
m5 = reg("learn ~ drop15 + y1 + y2 + y3 + y4 + a1 + a2 + a3 + a4")
m6 = reg("learn ~ drop15 + y1 + y2 + y3 + y4 + C(age)")

results = [m1, m2, m3, m4, m5, m6]
model_names = [
    "FS: YOB poly",
    "FS: +Age poly",
    "FS: Age FE",
    "RF: YOB poly",
    "RF: +Age poly",
    "RF: Age FE"
]

# -----------------------------------------------------
# Print summaries for each regression
# -----------------------------------------------------
for i, model in enumerate(results):
    print("\n==============================================")
    print(f"Summary for {model_names[i]}")
    print("==============================================\n")
    print(model.summary())

# -----------------------------------------------------
# Build clean output table like Table 1
# -----------------------------------------------------
table = pd.DataFrame({
    "Model": model_names,
    "Coef_drop15": [m.params["drop15"] for m in results],
    "SE_drop15": [m.bse["drop15"] for m in results]
})

print("\n==============================================")
print("           TABLE 1 (Great Britain)")
print("==============================================\n")
print(table)



Summary for FS: YOB poly

                            WLS Regression Results                            
Dep. Variable:               agelfted   R-squared:                       0.050
Model:                            WLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     4113.
Date:                Tue, 25 Nov 2025   Prob (F-statistic):           1.39e-41
Time:                        22:42:24   Log-Likelihood:                -63791.
No. Observations:               22574   AIC:                         1.276e+05
Df Residuals:                   22568   BIC:                         1.276e+05
Df Model:                           5                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.8329    

/Users/macbookpro/Documents/DSDM/Econometrics/assignment_7/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 38, but rank is 35
  warnings.warn('covariance of constraints does not have full '
/Users/macbookpro/Documents/DSDM/Econometrics/assignment_7/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 38, but rank is 34
  warnings.warn('covariance of constraints does not have full '
/Users/macbookpro/Documents/DSDM/Econometrics/assignment_7/.venv/lib/python3.13/site-packages/statsmodels/regression/linear_model.py:1884: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
